In [287]:
import pandas as pd

# Starting data sets

In [288]:
#The following are copies of the originally provided raw data from WBS
# orders.csv
orders = "https://drive.google.com/file/d/1gyoqm_HkCXYxPEAotoHy3AnjQb062db0/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+orders.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
orderlines = "https://drive.google.com/file/d/1jXT7JdxARmUrVc4qqE1wUPTuaEw7tjxw/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+orderlines.split("/")[-2]
orderlines = pd.read_csv(path)

# products.csv
products = "https://drive.google.com/file/d/1HUKPnuS9RJjrTstonYmTHDYGZ0foZbz9/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+products.split("/")[-2]
products = pd.read_csv(path)

# brands.csv
brands = 'https://drive.google.com/file/d/1C9fPQuinXOYkWe0NzcDgG3vLDg2seV1b/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+brands.split("/")[-2]
brands = pd.read_csv(path)

**Starting data status**

orders: (226909, 4)

orderlines:(293983, 7)

products:(19326, 7)

brands:(187, 2)

# Manipulation of data sets

### Removal of duplicates

In [290]:
#creating copies
orders_df = orders.copy()

orderlines_df = orderlines.copy()

products_df=products.copy()

brands_df=brands.copy()

In [291]:
# orders_df - no duplicates
orders_df.duplicated().sum()

0

In [292]:
# orderlines_df - no duplicates
orderlines_df.duplicated().sum()

0

In [293]:
#a very large amount of products are duplicated (8746, or 45%)
products_df.duplicated().sum()

8746

In [294]:
#dropping duplicates
products_df.drop_duplicates(inplace=True)

In [295]:
#verifying duplicates have been removed
#original number 19326 - -8746 removed = updated number 10580
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


In [296]:
#brands_df - no duplicates
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


**Data status after duplicates have been removed**

orders: (226909, 4) => (226909, 4)

orderlines:(293983, 7) ==> (293983, 7)

products:(19326, 7) ==> (10580, 7)

brands:(187, 2)==> (187, 2)

### Removal of missing values

In [298]:
#orders missing 5 values
orders_df.total_paid.isna().sum()

5

In [299]:
#orders missing values removed
orders_df = orders_df.loc[~orders_df.total_paid.isna(), :]

In [300]:
#orderlines has no missing values
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [301]:
#products has 3 columns missing values
# 7 values are missing from the desc column
# 46 values are missing from the price column
# 50 values are missing from the type column -- !!!BUT we will not remove it as it was not done officially by WBS!!!
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


In [302]:
#deleting desc rows with missing values
products_df.loc[products_df['desc'].isna(), 'desc'] = products_df.loc[products_df['desc'].isna(), 'name']

In [303]:
#deleting price rows with missing values
products_df = products_df.loc[~products_df.price.isna(), :]

**Data status after missing values have been removed**

orders: (226909, 4) => (226909, 4) ==> (226904, 4)

orderlines:(293983, 7) ==> (293983, 7) ==> (293983, 7)

products:(19326, 7) ==> (10580, 7) ==> (10534, 7)

brands:(187, 2)==> (187, 2) ==> (187, 2)

### Removal of two-dot problem (and column "promo_price")

Removal from orderlines DataFrame

In [305]:
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]
orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [306]:
orderlines_df.shape

(216250, 7)

In [307]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

Removals from products DataFrame

In [308]:
#looking at how many rows in products (column "price") have the two-dot problem = 542
price_problems_number = products_df.loc[(products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
price_problems_number

542

In [309]:
#removing the problem from products
products_df = products_df.loc[(~products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))&(~products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :]

In [310]:
#original count was 10580 - 588 removed = 9992 now
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9992 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          9992 non-null   object
 1   name         9992 non-null   object
 2   desc         9992 non-null   object
 3   price        9992 non-null   object
 4   promo_price  9992 non-null   object
 5   in_stock     9992 non-null   int64 
 6   type         9946 non-null   object
dtypes: int64(1), object(6)
memory usage: 624.5+ KB


In [311]:
#looking at how many rows in products (column "promo_price") have the two-dot problem  = 9232
promo_problems_number = products_df.loc[(products_df.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
promo_problems_number

9232

In [312]:
#over 90% of the values in promo_price are two-dotted
print(f"The column promo_price has in total {promo_problems_number} wrong values. This is {round(((promo_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column promo_price has in total 9232 wrong values. This is 92.39% of the rows of the DataFrame


In [313]:
#as over 90% of the data in the column is corrupt, complete column will be deleted

#completely deleting promo_price column
products_df = products_df.drop(columns=["promo_price"])

**Data status after two-dot problem has been removed**

orders: (226909, 4) => (226909, 4) ==> (226904, 4) ==> (226904, 4)

orderlines:(293983, 7) ==> (293983, 7) ==> (293983, 7) ==> (216250, 7)

products:(19326, 7) ==> (10580, 7) ==> (10534, 7) ==> (9992, 6)

brands:(187, 2)==> (187, 2) ==> (187, 2) ==> (187, 2)

### Removal of unwanted states

In [316]:
#making copies
o_df=orders_df.copy()
ol_df=orderlines_df.copy()
p_df=products_df.copy()
b_df=brands_df.copy()

In [317]:
#changing datatype
o_df["created_date"]=pd.to_datetime(o_df["created_date"])
ol_df["date"]=pd.to_datetime(ol_df["date"])

In [318]:
#keeping only rows with value "Completed" for column "state"
#original number of rows before filtering: 226904
#updated number of rows after filtering: 46605
o_df=o_df.query("state=='Completed'")

In [319]:
o_df.shape

(46605, 4)

**Data status after removal of unwanted states**

orders: (226909, 4) => (226909, 4) ==> (226904, 4) ==> (226904, 4) ==> (46605, 4)

orderlines:(293983, 7) ==> (293983, 7) ==> (293983, 7) ==> (216250, 7) ==> (216250, 7)

products:(19326, 7) ==> (10580, 7) ==> (10534, 7) ==> (9992, 6) ==> (9992, 6)

brands:(187, 2)==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2)

### Keeping only orders present in both orders and orderlines

In [321]:
orders_orderlines = o_df.merge(ol_df,
                left_on='order_id',
                right_on='id_order',
                how='inner')

In [322]:
#56714=original number of rows before removing order_id rows missing from the products table
orders_orderlines.shape

(56714, 11)

In [323]:
order_ids = orders_orderlines['order_id'].unique()

In [324]:
# keeping only the common order_ids in orders
o_df = o_df.loc[o_df["order_id"].isin(order_ids), :]

# keeping only the common order_ids in orderlines
ol_df = ol_df.loc[ol_df["id_order"].isin(order_ids), :]

In [325]:
#orders and orderlines now have same amount of unique order_ids
o_df["order_id"].nunique(), ol_df["id_order"].nunique()

(43064, 43064)

**Data status after keeping only orders present in both orders and orderlines**

orders: (226909, 4) => (226909, 4) ==> (226904, 4) ==> (226904, 4) ==> (46605, 4) ==> (43064, 4)

orderlines:(293983, 7) ==> (293983, 7) ==> (293983, 7) ==> (216250, 7) ==> (216250, 7) ==> (56714, 7)

products:(19326, 7) ==> (10580, 7) ==> (10534, 7) ==> (9992, 6) ==> (9992, 6) ==> (9992, 6)

brands:(187, 2)==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2)

### Removal of unknown products

In [327]:
orderlines_products = ol_df.merge(p_df, how="left", on = "sku")[["id_order","sku","name" ]]
orderlines_products.sample(10)

,id_order,sku,name
11505,351315,CRU0032,Crucial Mac Memory 8GB DDR3 1333MHz SO-DIMM
39552,459083,BEL0181,Belkin Thunderbolt Express HD Hub 2
44920,478423,SPH0021,Mini Sphero robotic ball Verde
38923,455519,DLK0061,D-Link DGS 1016D Gigabit Switch 16 Ports
55959,522980,APP1190,IPhone AppleCare Protection Plan
22747,393979,OWC0040-2,Mac OWC memory 16GB (2x8GB) SO-DIMM DDR3 1600MHZ
36601,433559,APP1643,Apple iPhone 7 Plus 32GB Rose Gold
34195,438635,SNS0020,Sonos Play 1 Bluetooth Speaker White
5988,317155,STI0009,Stil Mind iPhone Case Gentleman 6 / 6s Gray
3826,311295,KUA0009,Support Kukaclip car + iPhone Case SE / 5s / 5...


In [328]:
orders_to_delete = orderlines_products.loc[orderlines_products.name.isna(), "id_order"].unique()
orders_to_delete

array([299638, 299706, 299712, ..., 526106, 526122, 526363])

In [329]:
#keeping only the orders in orders and orderlines that are not corrupted

In [330]:
o_df = o_df.loc[~o_df.order_id.isin(orders_to_delete),:]

In [331]:
ol_df = ol_df.loc[~ol_df.id_order.isin(orders_to_delete),:]

In [332]:
o_df["order_id"].nunique(), ol_df["id_order"].nunique()

(41701, 41701)

**Data status after removal of unknown products**

orders: (226909, 4) => (226909, 4) ==> (226904, 4) ==> (226904, 4) ==> (46605, 4) ==> (43064, 4) ==> (41701, 4)

orderlines:(293983, 7) ==> (293983, 7) ==> (293983, 7) ==> (216250, 7) ==> (216250, 7) ==> (56714, 7) ==> (54324, 7)

products:(19326, 7) ==> (10580, 7) ==> (10534, 7) ==> (9992, 6) ==> (9992, 6) ==> (9992, 6) ==>  (9992, 6)

brands:(187, 2)==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2)

### Removal of outliers

In [334]:
#creating a new column with values of the other two tables multiplied
ol_df.loc[:,'unit_price_total'] = ol_df['unit_price'] * ol_df['product_quantity']

In [335]:
ol_agg = ol_df.groupby("id_order", as_index=False)["unit_price_total"].sum()

In [336]:
ol_agg_o = ol_agg.merge(o_df, left_on="id_order", right_on="order_id")

In [337]:
ol_agg_o_dif = ol_agg_o.copy()

In [338]:
#difference between total_paid and unit_price_total
ol_agg_o_dif["order_price_dif"] = ol_agg_o["total_paid"] - ol_agg_o["unit_price_total"]

In [339]:
# Removing outliers from ol_agg_o_diff (orderlines aggregated and merged with orders, dif between total_price and unit_price_total)

# Calculating Q1 (25th percentile) and Q3 (75th percentile)
Q1 = ol_agg_o_dif["order_price_dif"].quantile(0.25)
Q3 = ol_agg_o_dif["order_price_dif"].quantile(0.75)
IQR = Q3 - Q1

# Defining the outlier range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtering the DataFrame to remove outliers
ol_agg_o_dif_no_outliers = ol_agg_o_dif[(ol_agg_o_dif["order_price_dif"] >= lower_bound) & (ol_agg_o_dif["order_price_dif"] <= upper_bound)]

In [340]:
#Removing the outliers' unique order_id/id_order from orders as well as from orderlines_products merged DataFrame.
normal_diff_list = ol_agg_o_dif_no_outliers["id_order"]

In [341]:
o_df = o_df.loc[o_df["order_id"].isin(normal_diff_list),:]

In [342]:
ol_df = ol_df.loc[ol_df["id_order"].isin(normal_diff_list),:]

In [343]:
#40985, 40985
o_df["order_id"].nunique(), ol_df["id_order"].nunique()

(40985, 40985)

**Data status after removal of outliers**

orders: (226909, 4) => (226909, 4) ==> (226904, 4) ==> (226904, 4) ==> (46605, 4) ==> (43064, 4) ==> (41701, 4) ==> (40985, 4)

orderlines:(293983, 7) ==> (293983, 7) ==> (293983, 7) ==> (216250, 7) ==> (216250, 7) ==> (56714, 7) ==> (54324, 7) ==> (53231, 8)

products:(19326, 7) ==> (10580, 7) ==> (10534, 7) ==> (9992, 6) ==> (9992, 6) ==> (9992, 6) ==>  (9992, 6) ==> (9992, 6)

brands:(187, 2)==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2) ==> (187, 2)

# Cleaned (and categorized) data sets

In [192]:
# orders_clean_df.csv
orders_clean_df = "https://drive.google.com/file/d/1ZEuWF3-pKYC-i6fP8unAAG_JMqU2KEyc/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+orders_clean_df.split("/")[-2]
orders_clean_df = pd.read_csv(path)

# orderlines_clean_df.csv
orderlines_clean_df = "https://drive.google.com/file/d/1IbRAfLGhL7Yf_b8CVkknwhnzsdTBeCIt/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+orderlines_clean_df.split("/")[-2]
orderlines_clean_df = pd.read_csv(path)

# products_categorized_df.csv
products_categorized_df = "https://drive.google.com/file/d/12O5WsGIjbpwmKb-yU7ovn_UUNntvFOpa/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+products_categorized_df.split("/")[-2]
products_categorized_df = pd.read_csv(path)

# brands_clean_df.csv
brands_clean_df = "https://drive.google.com/file/d/1V33HMpSmSLjWSzmGv4is26QKIC_cqNpk/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+brands_clean_df.split("/")[-2]
brands_clean_df = pd.read_csv(path)